In [ ]:
# Dowload and install Ollama
!curl -fsSL https://ollama.com/install.sh | sh

## Start subprocess running ollama
If you are on a local machine you can simply run ```ollama serve```. This version is for shared notebooks as we wont have have root user permission to run directly.

In [9]:
import subprocess
subprocess.Popen(["ollama",])

<Popen: returncode: None args: ['ollama', 'serve']>

Error: listen tcp 127.0.0.1:11434: bind: address already in use


## Download Gemma
In this test we are using `gemma:2b`. Later we will want to replace this for our custom model but this is fine for testing.

In [ ]:
!ollama pull gemma:2b

In [ ]:
!pip install --upgrade --quiet openai pyautogen

In [13]:
import os
os.environ['OAI_CONFIG_LIST'] = '[{"model" : "gemma:2b", "api_key": "EMPTY", "max_tokens":1000}]'

In [18]:
import autogen

llm_config={
    "timeout": 600,
    "cache_seed": 68,  # change the seed for different trials
    "config_list": [{"model" : "gemma:2b", 
                     "api_key": "EMPTY", 
                     "max_tokens":1000,
                     "base_url":f"http://localhost:11434/v1"}],
    "temperature": 0.5,
}

## Define out agents
Here is where we will instantiate our AI agents. Right now we have the following agents outlined:

### User Agent
When the pipeline agent is complete this agent will be replaced with a recursive call for math problems. However in the meantime we will use it load individual questions one by one.

### Problem Manager
This agent will act as oversight, checking that the problem is first correctly broken down into a procedure, suggest edits or rounds of revision, to then be passed to the solver.

### Procedure Generator
This agent will be in charge of turning the question into a set of steps that will be used to procedurally solve the given math question. It will first need to read in the question, turn it into a summary goal, and then a set of steps for solving the problem.

### Mathmatician
This agent likely powered by deepseek will take a given procedure and apply it in an attempt to solve a math problem.



In [22]:
user_proxy = autogen.UserProxyAgent(
    name="User_proxy",
    system_message="A human admin.",
    code_execution_config=False,
    human_input_mode="TERMINATE",
    is_termination_msg=lambda x: x.get("content", "") and x.get("content", "").rstrip().endswith("TERMINATE"),
)
procedure = autogen.AssistantAgent(
    name="Procedure",
    system_message="""You an advanced AI acting as the an mathematics procedural assistant;
    you will take a given math problem written in LaTex format, create a high fidelity summary of what the problem is asking you to achieve;
    then break the task down into a list of individual steps that must be completed to solve the task.
                    """,
    llm_config=llm_config,
    description="""This is a math problem procedure assistant who is capable of taking a problem in LaTex format; summarizing the question; and creating a procedure for solving it.
                   The procedure assistant is open to any comments and recommendations for improving the summary and procedure.
                   Ask the procedure assistant to iterate every time when there is a new change recommendation from editor.
                """
)
editor = autogen.AssistantAgent(
    name="Editor",
    system_message="""You review the math questions and procedure proposals and give change recommendations to make the procedure and summary more aligned with the original question being asked,
    for example, revising to make sure the correct solution format is being protrayed.
                      You never write or revise procedures by yourself.
                       """,
    llm_config=llm_config,
    description="""This is an editor who reviews the procedure drafts of mathematics procedural assistants and provides change ideas.
                    The editor should be called every time the mathematics procedural assistant provides a version of a math procedure post.
                """
)

solver = autogen.AssistantAgent(
    name="Solver",
    system_message=""" You are an advanced mathematician;
    You will take a given math problem summary and a procedure for solving it;
    and you will follow the procedure to solve the math problem correctly.
                       """,
    llm_config=llm_config,
    description="""This is an elite math problem solver who reviews math procedures of procedure writing assistants and provides the solution to the problem.
                    The solver should be called every time the procedure assistant provides a version of procedure summary and steps.
                """
)
groupchat = autogen.GroupChat(agents=[user_proxy, procedure, editor, solver], messages=[], max_round=6)
manager = autogen.GroupChatManager(groupchat=groupchat, llm_config=llm_config)

In [47]:
import pandas as pd
from tqdm.notebook import tqdm
tqdm.pandas() # progress bar for pandas

df1 = pd.read_csv("traincompiled.csv")
df2 = pd.read_csv("testcompiled.csv")


df = pd.concat([df1, df2], axis=0)
df.head(2)

,Question,Level,Type,Solution,Answer
0,My school's math club has 6 boys and 8 girls. ...,Level 2,counting_and_probability,"With no restrictions, we are merely picking 6 ...",3003
1,How many 4-letter words with at least one cons...,Level 3,counting_and_probability,First we count the number of all 4-letter word...,609


In [48]:
template = """\n\nProblem:\n{problem}\n\nSolution:\n{solution}"""

df["prompt"] = df.progress_apply(lambda row: template.format(problem=row.Question,
                                                             solution=f"{row.Solution}\n\nAnswer:\n{row.Answer}"),
                                                             axis=1)
data = df.prompt.tolist()

  0%|          | 0/7929 [00:00<?, ?it/s]

In [53]:
q1 = df.head(1).Question.values [0]

# from IPython.display import display, Markdown

# def colorize_text(text):
#     for word, color in zip(["Role", "Instruction", "Problem", "Solution", "Answer"],
#                            ["blue", "yellow", "red", "cyan", "green"]):
#         text = text.replace(f"{word}:", f"\n\n**<font color='{color}'>{word}:</font>**")
#     return text

# # Take a random sample
# sample = data[12]

# display(data)

# Give colors to Instruction, Response and Category
# sample = colorize_text(sample)

# Show sample in markdown
# display(Markdown(sample))

In [54]:
user_proxy.initiate_chat(manager, message=q1)

User_proxy (to chat_manager):

My school's math club has 6 boys and 8 girls.  I need to select a team to send to the state math competition.  We want 6 people on the team.  In how many ways can I select the team without restrictions?

--------------------------------------------------------------------------------
Procedure (to chat_manager):

**Problem:**

How many ways can I select a team of 6 from a group of 14 students (6 boys and 8 girls)?

**Steps:**

1. Choose 6 boys from the 14 students. 
2. Choose 8 girls from the 14 students. 
3. Combine the 6 boys and 8 girls to form a team of 14. 

**Total ways to choose 6 boys and 8 girls from 14 students = 14C6 x 14C8 = 3003 ways.**

--------------------------------------------------------------------------------
Solver (to chat_manager):

The answer is 3003.

--------------------------------------------------------------------------------
Solver (to chat_manager):



-----------------------------------------------------------------------

ChatResult(chat_id=None, chat_history=[{'content': "My school's math club has 6 boys and 8 girls.  I need to select a team to send to the state math competition.  We want 6 people on the team.  In how many ways can I select the team without restrictions?", 'role': 'assistant'}, {'content': '**Problem:**\n\nHow many ways can I select a team of 6 from a group of 14 students (6 boys and 8 girls)?\n\n**Steps:**\n\n1. Choose 6 boys from the 14 students. \n2. Choose 8 girls from the 14 students. \n3. Combine the 6 boys and 8 girls to form a team of 14. \n\n**Total ways to choose 6 boys and 8 girls from 14 students = 14C6 x 14C8 = 3003 ways.**', 'name': 'Procedure', 'role': 'user'}, {'content': 'The answer is 3003.', 'name': 'Solver', 'role': 'user'}, {'content': '', 'name': 'Solver', 'role': 'user'}, {'content': 'The original problem asks for the number of ways to select a team of 6 from a group of 14 students (6 boys and 8 girls). The correct answer is 3003.\n\n**Explanation:**\n\nThe probl